# IBM Code Model Asset Exchange Image Segmenter

- https://github.com/IBM/MAX-Image-Segmenter


## Setup

1. In a terminal window, run the following commands to download and extract the model artifacts for the Image Segmenter:
    ```
    curl -O http://max-assets.s3-api.us-geo.objectstorage.softlayer.net/deeplab/deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz
    
    tar -zxvf deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz
    ```    

2. Run the notebook `jupyter notebook .`

In [ ]:
# This notebook has been tested with Python version 3.6
!python --version

In [ ]:
# This notebook has been tested with tensorflow 1.10.1, tensorflowjs 0.6.0, and numpy 1.14.5
!pip show tensorflow tensorflowjs numpy

In [ ]:
# Uncomment to install the packages needed

# !pip install -Iv tensorflow
# !pip install -Iv tensorflowjs
# !pip install -Iv numpy

# Restart the kernel after installation completes.

<br>

Update the variable with the appropriate directory path to extracted model

In [ ]:
# full path to extracted frozen graph
frozen_graph_path = '/Users/va/models/deeplabv3_mnv2_pascal_trainval_2018_01_29/frozen_inference_graph.pb'


<br>

Import libraries used in this notebook

In [ ]:
from PIL import Image
import numpy as np
import pathlib
import os
import time
import tensorflow as tf

In [ ]:
print('TF versions:', tf.GIT_VERSION, tf.VERSION)


<br>

## Load frozen graph

In [ ]:
# load the frozen file and parse it to get the unserialized graph_def
def load_frozen_graph(graph_path):
    with tf.gfile.GFile(graph_path, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        return graph_def

In [ ]:
# load graph
restored_graph_def = load_frozen_graph(frozen_graph_path)



<br>

## Inspect the graph

In [ ]:
# print list graph nodes/tensors
def list_nodes(graph_def):
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            restored_graph_def,
            input_map=None,
            return_elements=None,
            name=""
        )

    sess = tf.Session(graph=graph)
    nodes = sess.graph.as_graph_def().node
    print('graph has {} nodes \r\n'.format(len(nodes)))
    
    for n in nodes:
        print(n.name + '=>' +  n.op)


In [ ]:
# list graph nodes
list_nodes(restored_graph_def)



<br>

## Run inference

<br>

Set the helper functions

In [ ]:

INPUT_TENSOR_NAME = 'ImageTensor:0'
OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
# value to resize image
IMAGE_SIZE = 512

# resize the image
def resize_image(image_path):
    image = Image.open(image_path)

    width, height = image.size
    resize_ratio = 1.0 * IMAGE_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized = image.convert('RGB').resize(target_size, Image.ANTIALIAS)

    return resized


# run prediction
def run_inference(graph_def, image):
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            restored_graph_def,
            input_map=None,
            return_elements=None,
            name=""
        )

    sess = tf.Session(graph=graph)

    batch_seg_map = sess.run(
        OUTPUT_TENSOR_NAME,
        feed_dict = { INPUT_TENSOR_NAME: [np.asarray(resized_image)] }
    )
    
    return batch_seg_map[0]


# Creates a label colormap used in PASCAL VOC segmentation benchmark
def create_pascal_label_colormap():
    colormap = np.zeros((256, 3), dtype=int)
    ind = np.arange(256, dtype=int)

    for shift in reversed(range(8)):
        for channel in range(3):
            colormap[:, channel] |= ((ind >> channel) & 1) << shift
        ind >>= 3

    return colormap


# Adds color defined by the dataset colormap to the label
def label_to_color_image(label):
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    colormap = create_pascal_label_colormap()

    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')

    return colormap[label]

<br>

Set the path of the image to use and display the resized image. You can use any image.

In [ ]:
image_path = '/Users/va/models/stc.jpg'

resized_image = resize_image(image_path)

resized_image

<br>

Run prediction for the image

In [ ]:
# run prediction
seg_map = run_inference(restored_graph_def, resized_image)
print (seg_map)


<br>

Map individual segment results to a color and display

In [ ]:

# map results to color
seg_image = label_to_color_image(seg_map).astype(np.uint8)

# display results
Image.fromarray(seg_image)

<br>
<hr>

# Converting to a web-friendly format

[https://github.com/tensorflow/tfjs-converter](https://github.com/tensorflow/tfjs-converter)


```
tensorflowjs_converter \
    --input_format=tf_frozen_model \
    --output_node_names='SemanticPredictions' \
    /path/to/frozen/model.pb \
    /path/to/web_asset_output_dir
```


In [ ]:
# full path to directory where converter output will be saved
web_asset_dir = '/Users/va/models/web_assets'

# create directory if it does not exist
pathlib.Path(web_asset_dir).mkdir(parents=True, exist_ok=True)

Run converter

In [ ]:
# set appropriate frozen model path and desired output path for web format

!tensorflowjs_converter \
    --input_format=tf_frozen_model \
    --output_node_names='SemanticPredictions' \
    {frozen_graph_path} \
    {web_asset_dir}


In [ ]:
print("Web asset directory {}:".format(web_asset_dir))

web_assets = os.listdir(web_asset_dir)
web_assets.sort()

for file in web_assets:
    file_stat = os.stat("{}/{}".format(web_asset_dir,file))
    print(" {} {} {:>20}".format(file.ljust(30), time.ctime(file_stat.st_mtime), file_stat.st_size))
